In [3]:
# Animated Earthquake Visualization (Plotly) — Artistic, dazzling animation
# 运行环境: Jupyter Notebook (推荐) 或者 Python 脚本（会生成 HTML）

import requests
import pandas as pd
import plotly.express as px
from datetime import datetime, timezone
import math

# 1) 下载 GeoJSON（USGS all_month）
URL = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson"
r = requests.get(URL, timeout=20)
r.raise_for_status()
data = r.json()

# 2) 把 GeoJSON 转成 DataFrame（每个 feature 的 properties + coords）
rows = []
for feat in data.get("features", []):
    props = feat.get("properties", {})
    geom = feat.get("geometry", {})
    coords = geom.get("coordinates", [None, None, None])  # [lon, lat, depth]
    lon, lat, depth = coords[0], coords[1], coords[2]
    # properties: time (ms since epoch), mag, place, url, etc.
    time_ms = props.get("time")
    if time_ms is None:
        continue
    dt = datetime.fromtimestamp(time_ms / 1000.0, tz=timezone.utc)  # UTC datetime
    rows.append({
        "time": dt,
        "time_ts": int(time_ms/1000),   # seconds
        "longitude": lon,
        "latitude": lat,
        "depth": depth,
        "mag": props.get("mag") if props.get("mag") is not None else 0.0,
        "place": props.get("place"),
        "id": feat.get("id")
    })

df = pd.DataFrame(rows)
if df.empty:
    raise SystemExit("No earthquake data found in feed.")

# 3) 预处理：清理异常值，填补、转类型
df['mag'] = pd.to_numeric(df['mag'], errors='coerce').fillna(0.0)
df['depth'] = pd.to_numeric(df['depth'], errors='coerce').fillna(0.0)

# 4) 分帧策略：按日（你也可以改成 'hour'）
# 添加 'frame' 列，例如 '2025-09-01' 形式
df['frame'] = df['time'].dt.strftime('%Y-%m-%d')  # day frames
# 或按小时: df['frame'] = df['time'].dt.strftime('%Y-%m-%d %H:00')

# 5) 要做更平滑、更艺术的动画，可以生成"fading"多重帧（可选）
#    这里我们先做单帧：每帧显示那天所有地震；颜色映射 mag，大小映射 mag
#    然后我们设置动画中的 transition / frame duration 来获得流畅感觉.

# 6) Size scaling: 让震级映射到点大小（艺术化）
def mag_to_size(m):
    # 非线性放大：以 2**mag 为参考再缩放
    return (2 ** (m/1.5)) * 2.0

df['size'] = df['mag'].apply(lambda m: mag_to_size(m))

# 7) Color scale: use a colourful sequential / diverging scale
color_continuous_scale = "Magma"  # 很漂亮的色带，或者 'Turbo', 'Viridis'

# 8) 绘图（Plotly Express scatter_geo with animation_frame）
fig = px.scatter_geo(
    df,
    lon='longitude',
    lat='latitude',
    color='mag',
    size='size',
    hover_name='place',
    hover_data={'mag': True, 'depth': True, 'time': True, 'longitude': False, 'latitude': False, 'size': False},
    projection="natural earth",
    animation_frame='frame',
    title='Earthquakes (past 30 days) — animated by day',
    color_continuous_scale=color_continuous_scale,
    range_color=[df['mag'].min(), df['mag'].max()],
    height=700,
)

# 9) 美化：去掉海洋/大陆默认样式，调动画速度与过渡
fig.update_layout(
    geo=dict(
        showland=True, landcolor="rgb(17, 24, 34)",  # 深色土地
        showocean=True, oceancolor="rgb(5,10,25)",   # 深色海洋
        bgcolor="black"
    ),
    paper_bgcolor="black",
    plot_bgcolor="black",
    coloraxis_colorbar=dict(title="Magnitude"),
    font=dict(color="white"),
)

# 让点有发光/半透明的视觉效果（通过 marker.opacity + marker.line）
fig.update_traces(
    marker=dict(opacity=0.8, line=dict(width=0.5, color='rgba(255,255,255,0.2)')),
)

# 10) 动画控制：帧速度（ms）
# 你可以在 Notebook 里直接交互式播放，也可以设置默认动画参数
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 600  # 每帧 600ms
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 300

# 11) 显示或导出
fig.show()   # Notebook 内部将直接显示交互动态图
# 或保存为单文件 HTML:
fig.write_html("earthquakes_anim.html", include_plotlyjs='cdn')
print("Saved earthquakes_anim.html")

# 额外扩展（如果你想要残影/拖尾效果，需要更复杂的数据扩展处理）


Saved earthquakes_anim.html
